# Broadcast Communication

By default, communication occurs between all satellites that are specified by the communication
method. This tutorial shows how to use two classes to configure a broadcast action that must be
taken for communication to occur:

* [Broadcast](../api_reference/act/index.html#bsk_rl.act.Broadcast), which gives satellites an action
  that enables communication from them at the end of the current step.
* [BroadcastCommunication](../api_reference/comm/index.html#bsk_rl.comm.BroadcastCommunication), which
  can be combined with another communication method to limit communication from broadcasters to
  those satellites satisfying the requirements of the other communication method.

## Configuring the Environment

For this example, a multisatellite target imaging environment will be used. The goal is
to maximize the value of unique images taken. This configuration is similar to the 
[Multi-Agent Environments](../examples/multiagent_envs.html) example.

In [ ]:
from bsk_rl import sats, act, obs, scene, data, comm
from bsk_rl.sim import dyn, fsw


class ImagingSatellite(sats.ImagingSatellite):
    observation_spec = [
        obs.OpportunityProperties(
            dict(prop="priority"),
            dict(prop="opportunity_open", norm=5700.0),
            n_ahead_observe=4,
        )
    ]
    action_spec = [act.Broadcast(duration=15.0), act.Image(n_ahead_image=4)]
    dyn_type = dyn.FullFeaturedDynModel
    fsw_type = fsw.SteeringImagerFSWModel


ACTION_BROADCAST = 0
ACTION_IMAGE_0 = 1
ACTION_IMAGE_1 = 2
ACTION_IMAGE_2 = 3
ACTION_IMAGE_3 = 4

Satellite properties are set to give the satellite near-unlimited power and storage resources. To randomize some parameters in a correlated manner across satellites, a ``sat_arg_randomizer`` is set and passed to the environment. In this case, the satellites are distributed in a trivial single-plane Walker-delta constellation.

In [ ]:
from bsk_rl.utils.orbital import walker_delta_args

N_AGENTS = 2
sat_args = dict(
    imageAttErrorRequirement=0.01,
    imageRateErrorRequirement=0.01,
    batteryStorageCapacity=1e9,
    storedCharge_Init=1e9,
    dataStorageCapacity=1e12,
    u_max=0.4,
    K1=0.25,
    K3=3.0,
    omega_max=0.087,
    servo_Ki=5.0,
    servo_P=150 / 5,
)
sat_arg_randomizer = walker_delta_args(
    altitude=800.0, inc=60.0, n_planes=1, clustersize=N_AGENTS, clusterspacing=5.0
)

A communication type is defined that uses multidegree line-of-sight communication with broadcasting.

In [ ]:
class BroadcastLOS(comm.BroadcastCommunication, comm.LOSMultiCommunication):
    pass

Finally, the environment can be instantiated.

In [ ]:
from bsk_rl import ConstellationTasking

env = ConstellationTasking(
    satellites=[ImagingSatellite(f"EO-{i + 1}", sat_args) for i in range(N_AGENTS)],
    scenario=scene.UniformTargets(1000),
    rewarder=data.UniqueImageReward(),
    communicator=BroadcastLOS(),
    sat_arg_randomizer=sat_arg_randomizer,
    log_level="INFO",
)
_ = env.reset()

In the first step, both agents are tasked with an imaging action, and one successfully images a target.

In [ ]:
_ = env.step({"EO-1": ACTION_IMAGE_3, "EO-2": ACTION_IMAGE_2})

When both select the broadcast action, data is shared in both directions. This is subject to line-of-sight availability as well as selecting the correct action.

In [ ]:
_ = env.step({"EO-1": ACTION_BROADCAST, "EO-2": ACTION_BROADCAST})

One agent can broadcast while the others are completing a different task.

In [ ]:
_ = env.step({"EO-1": ACTION_IMAGE_2, "EO-2": ACTION_BROADCAST})  # Agent 1 broadcasts